### Kaggle: Avito Demand Prediction Challenge
source: https://www.kaggle.com/c/avito-demand-prediction


Intro: <font color="red"> criar descrição</font>


Team: <font color="red"> criar nome</font>
- Luis Filipe Kopp (lfkopp)
- Lucas Pérez (lucascperez)
- Raphael Reis (raphacoelho)

# Part two - generating bag of words 


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from collections import Counter
from nltk import download
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [2]:
%matplotlib inline
dir = 'c:/_dados/_avito/'
download('punkt')
stemmer = SnowballStemmer("russian")

stop2 = [u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок']
stop = set([x.lower() for x in (stopwords.words('russian') + stop2)])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lfkop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def remove_emoji(text):
    emoji_pattern = re.compile("[🦋⭐！�����🥀⤴️🤘🦄🤰🤚🆗⬛️⭕️🦉⬆️⬆⬇🤸🆕🤣🤔🤓🤗🥅🆘🦁$〰〽️駅伝競走ஜ۩۞۩ஜ#🤷🦊\U00000025-\U00000150\U00002000-\U00002800\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

def convert(frase):
    frase = remove_emoji(frase)
    frase = frase.replace(str(np.NaN), ' ').replace('"', ' ').replace('!', ' ').replace('#', ' ')
    frase = re.sub("\s\s+", " ", frase)
    return ' '.join([stemmer.stem(word).lower() for word in str(frase).split() if ((word not in stop) and (len(word)>2))])

In [3]:
df = pd.read_pickle(dir + 'df2.pickle')
display(df.head())
display(df.shape)

,activation_date,category_name,city,deal_probability,deal_rounded,description,image_top_1,item_id,month,param_1,...,sold_0,sold_1,sold_2,sold_3,sold_4,sold_5,sold_6,sold_7,sold_8,sold_9
0,2017-03-28,Товары для детей и игрушки,Екатеринбург,0.12789,0.2,"Кокон для сна малыша,пользовались меньше месяц...",1008.0,b912c3c6a6ad,3,Постельные принадлежности,...,0,0,0,0,0,0,1,0,0,0
1,2017-03-26,Мебель и интерьер,Самара,0.00000,0.0,"Стойка для одежды, под вешалки. С бутика.",692.0,2dac0150717d,3,Другое,...,0,0,0,0,0,0,0,0,0,0
2,2017-03-20,Аудио и видео,Ростов-на-Дону,0.43177,0.4,"В хорошем состоянии, домашний кинотеатр с blu ...",3032.0,ba83aefab5dc,3,"Видео, DVD и Blu-ray плееры",...,0,0,0,0,0,0,1,0,1,1
3,2017-03-25,Товары для детей и игрушки,Набережные Челны,0.80323,0.8,Продам кресло от0-25кг,796.0,02996f1dd2ea,3,Автомобильные кресла,...,1,1,1,1,1,1,0,0,1,1
4,2017-03-16,Автомобили,Волгоград,0.20797,0.2,Все вопросы по телефону.,2264.0,7c90be56d2ab,3,С пробегом,...,0,0,0,0,0,1,0,0,0,0


(2011862, 32)

In [4]:
groups = df.parent_category_name.unique()
for i,group in enumerate(groups):
    print(i,group)  

0 Личные вещи
1 Для дома и дачи
2 Бытовая электроника
3 Транспорт
4 Недвижимость
5 Животные
6 Хобби и отдых
7 Услуги
8 Для бизнеса


In [6]:
df2 = df[['item_id', 'parent_category_name', 'source', 'deal_probability', 'title', 'description']]
for i,group in enumerate(groups):
    df3 = df2[df2['parent_category_name'] == group]
    #df3 = df3[df3['source'] == 'train']
    with open(dir+'consolidated_'+str(i)+'.txt', 'wb') as f: 
        with open(dir + 'stemmed_'+str(i)+'.txt', 'wb') as s: 
            for line, row in df3.iterrows():
                content = ''
                idx = row.item_id
                desc = convert(str(row.title) + ' ' + str(row.description)).split()
                deal = str(np.round(row.deal_probability,5))
                src = str(row.source)
                con = Counter(desc).items()
                st = str(group) + ';' +str(idx) + ';' +str(src) + ';' +str(deal) + ';'
                st2 = []
                for item in con:
                    a,b = item
                    content += str(group) + ';' +str(idx) + ';' +str(src) + ';' +str(deal) + ';' + str(a) + ';' + str(b) + "\r\n"
                    st2.append(a)
                st += ' '.join(list(set(st2)))
                st += "\r\n"
                f.write(content.encode('utf-8'))
                s.write(st.encode('utf-8'))

In [8]:
for i,group in enumerate(groups):
    df5 = pd.read_csv(dir + 'consolidated_'+str(i)+'.txt', sep=';', names=['parent_category', 'item_id', 'source', 'deal_probability', 'word_stem', 'number'])
    df6 = pd.read_csv(dir + 'stemmed_'+str(i)+'.txt', sep=';', names=['parent_category', 'item_id', 'source', 'deal_probability', 'phrase_stem'])
    df5['greater'] = df5.deal_probability > 0.3
    df5['less'] = df5.deal_probability < 0.3
    dw = pd.DataFrame(df5[df5.source == 'train'].groupby('word_stem')[['number','deal_probability','greater','less']].sum())
    dw['perc_greater'] = dw.greater / (dw.greater + dw.less)
    dw.sort_values(by='number', ascending=False, inplace=True)
    dw = dw[:8000]
    dw['avg_deal'] = np.round(dw.deal_probability / dw.number ,4)
    mean = dw['avg_deal'].mean()
    std = dw['avg_deal'].std()
    dw['avg_deal_normal'] = (dw['avg_deal']-mean)/std
    dw['avg_deal_normal_sq'] = dw['avg_deal_normal'] ** 2
    dw.sort_values(by='avg_deal_normal_sq', ascending=False, inplace=True)
    dw = dw[:4000]
    dw.to_pickle(dir + 'dw_' + str(i) + '.pickle')
    words = dw.index.tolist()
    with open(dir+'matrix_'+str(i)+'.txt', 'wb') as f:
        f.write(('item_id;source;deal_probability;'+';'.join(words) + "\r\n").encode('utf-8'))
        for j,l in df6.iterrows():
            linha = str(l.item_id) + ';' + str(l.source) + ';' + str(np.round(l.deal_probability,5)) 
            v = str(l.phrase_stem).split()
            for word in words:
                linha += ';'+ str((word.lower() in v)*1)
            linha += "\r\n"
            f.write(linha.encode('utf-8'))



In [8]:
for i,group in enumerate(groups):
    print(i,group)
    dt = pd.read_csv(dir + 'matrix_'+ str(i)+'.txt', sep=';', encoding='utf-8', nrows=10)
    for j,c in enumerate(dt.columns):
        if j>2: dt[c] = dt[c].astype(bool)
    datatype = dict(dt.dtypes)
    dt = pd.read_csv(dir + 'matrix_'+ str(i)+'.txt', sep=';', encoding='utf-8', dtype=datatype)
    display(dt.head())
    display(dt.shape)
    dt.to_pickle(dir + 'matrix_' + str(i) + '.pickle')
    

0 Личные вещи


,item_id,source,deal_probability,подносик,расшата,полулежач,псевдорезин,уснул,карзин,полусид,...,ема,исследован,горнолыжк,плавочк,инфракрасн,молод,могл,глубок,зрен,выросл
0,b912c3c6a6ad,train,0.12789,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,02996f1dd2ea,train,0.80323,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,51e0962387f7,train,0.80323,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,6b71309d6a8a,train,0.80323,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,c5b969cb63a2,train,0.00000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


(914200, 4003)

1 Для дома и дачи


,item_id,source,deal_probability,вынест,печет,самовынос,потрепа,электророзжиг,чистеньк,идел,...,антик,возникнут,планет,влагостойк,меньш,эма,парм,стенов,подверга,термометр
0,2dac0150717d,train,0.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,c4f260a2b48a,train,0.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,78164bc09657,train,0.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,af9d5b0eadc9,train,0.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,75076ea12736,train,0.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


...

,item_id,source,deal_probability,шаланд,низкорамник,ивек,стеклоподъемник,индез,духовок,зубренок,...,состо,поездк,соц,рассчита,гарант,большинств,лакировк,уведомлен,справедлив,дор
0,fc21a86b9bb0,train,0.80,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,92013ca1fe79,train,1.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,c6239fc67a6f,train,1.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1298ab2ff7f3,train,0.50,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,83f15342ba4c,train,0.75,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


(88004, 4003)

8 Для бизнеса


,item_id,source,deal_probability,футболок,бедр,горд,хобб,сухожар,недолг,неработа,...,хрустал,переоборудован,дорогостоя,полис,самоклеющ,образова,радиолифтинг,инв,нерж,тефлон
0,b4a030416810,train,0.78912,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,a968a9480a7c,train,0.00000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,f840a7c7fbc6,train,0.12267,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,f24aa3aed062,train,0.00000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,d97207839dbf,train,0.62265,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


(24538, 4003)

In [ ]:
#d['used'] = 'б\у'
#d['forfree'] = 'бесплатно'